In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors

from exp_spec_info import *
from plot_info import *

In [2]:
# Data pickle path
extracted_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extracted_data.pkl"

# Processed data paths
processed_data_root = "C:\\Users\\dosre\\dev\\thesis-data\\processed_data"
os.makedirs(processed_data_root, exist_ok=True)
plots_7_1 = os.path.join(processed_data_root, "plots_7_1")
os.makedirs(plots_7_1, exist_ok=True)
# conv_traj_dir = os.path.join(processed_data_root, "nonlog_convergence_trajectories")
# os.makedirs(conv_traj_dir, exist_ok=True)


In [3]:
data = pd.read_pickle(extracted_data_path)

In [4]:
def df_sel_setup(df, setup) -> pd.DataFrame:
    return df[df["setup"] == setup]

def df_sel_setup_matrix(df, setup, matrix) -> pd.DataFrame:
    return df[(df["setup"] == setup) & (df["matrix"] == matrix)]

def df_sel_setup_matrix_inneriter(df, setup, matrix, inneriter) -> pd.DataFrame:
    return df[
        (df["setup"] == setup) &
        (df["matrix"] == matrix) &
        (df["inner_iter"] == inneriter)
    ]

In [5]:
conv_traj_data = data[
    (data["solver"] == "FP FP16") |
    (data["solver"] == "FP FP32") |
    (data["solver"] == "FP FP64")
]

relres_diff_data = {
    "FP FP16": [],
    "FP FP32": []
}

first_deriv_data = {
    "FP FP16": [],
    "FP FP32": []
}

for setup in SETUPS:
    for matrix in df_sel_setup(conv_traj_data, setup)["matrix"].unique():
            for inner_iter in df_sel_setup_matrix(conv_traj_data, setup, matrix)["inner_iter"].unique():

                sub_data = df_sel_setup_matrix_inneriter(
                    conv_traj_data, setup, matrix, inner_iter
                )

                for iter in sub_data["experiment_iter"].unique():

                    for solver in ["FP FP16", "FP FP32"]:

                        lowfp_data = sub_data[
                            (sub_data["solver"] == solver) &
                            (sub_data["experiment_iter"] == iter)
                        ]

                        # Calculate first derivative
                        if lowfp_data.size != 0:
                            lowfp_data_row = lowfp_data.iloc[0]["inner_relres"]
                            fm1 = lowfp_data_row[:(lowfp_data_row.size-2)]
                            f1 = lowfp_data_row[2:]
                            deriv_to_add = np.hstack([
                                (lowfp_data_row[1]-lowfp_data_row[0]),
                                0.5*(f1-fm1),
                                (lowfp_data_row[-1]-lowfp_data_row[-2])
                            ])
                            if 15001-deriv_to_add.size > 0:
                                deriv_to_add = np.hstack(
                                    (deriv_to_add,
                                     np.full(15001-deriv_to_add.size, np.nan))
                                )
                            
                            # Flip deriv and remove any increased
                            deriv_to_add *= -1.
                            deriv_to_add[deriv_to_add <= 1e-12] = 1e-12

                            first_deriv_data[solver].append(deriv_to_add)
                            

                        # Calculate relres diff between lowfp and fp64
                        fp64_data = sub_data[
                            (sub_data["solver"] == "FP FP64") &
                            (sub_data["experiment_iter"] == iter)
                        ]
                        if lowfp_data.size != 0:
                            kwargs = {
                                "color": SOLVER_CLR_DICT[solver],
                                "alpha": 0.005,
                                "linestyle": None,
                                "marker": "."
                            }
                            lowfp_data_row = lowfp_data.iloc[0]["inner_relres"]
                            control_data_row = fp64_data.iloc[0]["inner_relres"]
                            if control_data_row.size < lowfp_data_row.size :
                                lowfp_data_row = lowfp_data_row[:control_data_row.size ]
                            elif control_data_row.size > lowfp_data_row.size:
                                control_data_row = control_data_row[:lowfp_data_row.size]
                            
                            diff_data_to_add = lowfp_data_row-control_data_row
                            diff_data_to_add = np.append(
                                diff_data_to_add,
                                np.full(15001-diff_data_to_add.size, np.nan)
                            )
                            
                            diff_data_to_add[diff_data_to_add < 1e-12] = 1e-12
                            
                            relres_diff_data[solver].append(diff_data_to_add)

relres_diff_data["FP FP16"] = np.vstack(relres_diff_data["FP FP16"])
relres_diff_data["FP FP32"] = np.vstack(relres_diff_data["FP FP32"])

diff_max_val = np.nanmax(np.hstack((relres_diff_data["FP FP16"], relres_diff_data["FP FP32"])))
diff_min_val = np.nanmin(np.hstack((relres_diff_data["FP FP16"], relres_diff_data["FP FP32"])))

first_deriv_data["FP FP16"] = np.vstack(first_deriv_data["FP FP16"])
first_deriv_data["FP FP32"] = np.vstack(first_deriv_data["FP FP32"])

deriv_max_val = np.nanmax(np.hstack((first_deriv_data["FP FP16"], first_deriv_data["FP FP32"])))
deriv_min_val = np.nanmin(np.hstack((first_deriv_data["FP FP16"], first_deriv_data["FP FP32"])))

##### 7.1 Wasted Work Characterization Relres Diff Plots

In [6]:
for solver in ["FP FP16", "FP FP32"]:

    fig, axs = plt.subplots(1, 3, figsize=(6.4, 6.4), width_ratios=[0.49, 0.49, 0.02])
    ax1, ax2, ax3 = axs

    iter_split = 200
    max_iters = relres_diff_data[solver].shape[1]
    indices_iters = np.arange(0, max_iters, 1)
    indices_exp = np.arange(0, relres_diff_data[solver].shape[0], 1)
    X, Y = np.meshgrid(indices_iters, indices_exp)

    ax1.pcolormesh(
        X[:, :iter_split], Y[:, :iter_split],
        relres_diff_data[solver][:, :iter_split],
        cmap="RdYlGn_r",
        norm=matplotlib.colors.LogNorm(vmin=diff_min_val, vmax=diff_max_val)
    )
    ax1.set_xticks(np.arange(0, iter_split+1, 25))
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45)
    ax1.set_xlabel("Inner Iterations")
    ax1.set_yticks([])
    ax1.set_ylabel("Linear Solve Experiment")

    ax_im = ax2.pcolormesh(
        X[:, iter_split:], Y[:, iter_split:],
        relres_diff_data[solver][:, iter_split:],
        cmap="RdYlGn_r",
        norm=matplotlib.colors.LogNorm(vmin=diff_min_val, vmax=diff_max_val)
    )
    ax2.set_xticks([iter_split] + list(np.arange(2500*int(iter_split/2500+1), max_iters+1, 2500)))
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45)
    ax2.set_xlabel("Inner Iterations")
    ax2.set_yticks([])

    fig.suptitle(f"{solver.replace("FP ", "")} Relative Residual Diff. From FP64")
    fig.colorbar(ax_im, cax=ax3)
    fig.tight_layout()

    plt.savefig(os.path.join(plots_7_1, f"wasted_work_{solver.replace("FP ", "")}.png"), bbox_inches='tight')
    plt.close()

##### 7.1 Stagnation Characterization

In [7]:
for solver in ["FP FP16", "FP FP32"]:

    fig, axs = plt.subplots(1, 3, figsize=(6.4, 6.4), width_ratios=[0.49, 0.49, 0.02])
    ax1, ax2, ax3 = axs

    iter_split = 200
    max_iters = relres_diff_data[solver].shape[1]
    indices_iters = np.arange(0, max_iters, 1)
    indices_exp = np.arange(0, relres_diff_data[solver].shape[0], 1)
    X, Y = np.meshgrid(indices_iters, indices_exp)

    ax1.pcolormesh(
        X[:, :iter_split], Y[:, :iter_split],
        first_deriv_data[solver][:, :iter_split],
        cmap="RdYlGn",
        norm=matplotlib.colors.LogNorm(vmin=deriv_min_val, vmax=deriv_max_val)
    )
    ax1.set_xticks(np.arange(0, iter_split+1, 25))
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45)
    ax1.set_xlabel("Inner Iterations")
    ax1.set_yticks([])
    ax1.set_ylabel("Linear Solve Experiment")

    ax_im = ax2.pcolormesh(
        X[:, iter_split:], Y[:, iter_split:],
        first_deriv_data[solver][:, iter_split:],
        cmap="RdYlGn",
        norm=matplotlib.colors.LogNorm(vmin=deriv_min_val, vmax=deriv_max_val)
    )
    ax2.set_xticks([iter_split] + list(np.arange(2500*int(iter_split/2500+1), max_iters+1, 2500)))
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45)
    ax2.set_xlabel("Inner Iterations")
    ax2.set_yticks([])

    fig.suptitle(f"{solver.replace("FP ", "")} Negative Relative Residual First Derivative")
    fig.colorbar(ax_im, cax=ax3)
    fig.tight_layout()

    plt.savefig(os.path.join(plots_7_1, f"stagnation_{solver.replace("FP ", "")}.png"), bbox_inches='tight')
    plt.close()